# INFOGAN with metrics

* souce: https://github.com/openai/InfoGAN/tree/master

In [1]:
#!pip install prettytensor

In [2]:
#!pip install progressbar

In [3]:
#!pip install python-dateutil

In [4]:
from __future__ import annotations
import os, math, json, errno, time
from pathlib import Path
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.linalg import sqrtm
import tensorflow as tf
from tensorflow.keras import layers, Model
from PIL import Image

# ----------------------------------------------------------------------------
# Utils ----------------------------------------------------------------------
# ----------------------------------------------------------------------------
TINY = 1e-8
floatX = np.float32

def mkdir_p(path: str):
    Path(path).mkdir(parents=True, exist_ok=True)


# Dataset loader (CelebA)
class CelebADataset:
    """Loader simplificado da CelebA sem atributos."""

    def __init__(self, root_dir: str = '.', image_shape=(64, 64, 3), split_ratio=0.9):
        self.image_shape = image_shape
        self.image_dim = int(np.prod(image_shape))
        base = Path(root_dir)

        # Encontra todas as imagens no diretório
        img_dir = base / 'img_align_celeba'
        sub = img_dir / 'img_align_celeba'
        if sub.is_dir():
            img_dir = sub
            
        self.files = sorted([f for f in img_dir.glob('*.jpg')])
        
        # Split train
        n_train = int(len(self.files) * split_ratio)
        self.train_idx = np.arange(n_train)
        np.random.shuffle(self.train_idx)
        self.ptr = 0

    def next_batch(self, batch_size):
        if self.ptr + batch_size > len(self.train_idx):
            np.random.shuffle(self.train_idx)
            self.ptr = 0
        sel = self.train_idx[self.ptr:self.ptr + batch_size]
        self.ptr += batch_size
        imgs = []
        for i in sel:
            img = Image.open(self.files[i]).resize(self.image_shape[:2])
            imgs.append(np.asarray(img, np.float32) / 127.5 - 1.0)
        x = np.stack(imgs).reshape(batch_size, -1)
        return x

    def inverse_transform(self, flat):
        imgs = flat.reshape((-1,) + self.image_shape)
        return ((imgs + 1.) * 127.5).clip(0, 255).astype(np.uint8)


# Distributions 
import tensorflow as tf
import numpy as np
from typing import Dict, List, Union

TINY = 1e-8
floatX = np.float32

class Distribution:
    """Classe base abstrata para distribuições latentes no InfoGAN."""
    
    @property
    def dim(self) -> int:
        """Dimensão da variável aleatória (tamanho do vetor de saída)."""
        raise NotImplementedError
        
    @property
    def dist_flat_dim(self) -> int:
        """Dimensão do vetor plano de parâmetros da distribuição."""
        raise NotImplementedError
        
    @property
    def effective_dim(self) -> int:
        """Dimensão efetiva para cálculo de mutual information."""
        return self.dim
        
    def logli(self, x_var: tf.Tensor, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        """
        Calcula o log-likelihood log p(x|θ) para cada amostra no batch.
        """
        raise NotImplementedError
        
    def sample(self, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        """
        Gera amostras da distribuição parametrizada por dist_info.
        """
        raise NotImplementedError
        
    def sample_prior(self, batch_size: int) -> tf.Tensor:
        """Gera amostras da distribuição prévia (prior)."""
        return self.sample(self.prior_dist_info(batch_size))
        
    def prior_dist_info(self, batch_size: int) -> Dict[str, tf.Tensor]:
        """
        Retorna os parâmetros da distribuição prévia (prior).
        """
        raise NotImplementedError
        
    def entropy(self, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        """
        Calcula a entropia H[p(x|θ)] para cada amostra no batch.
        """
        raise NotImplementedError
        
    def marginal_entropy(self, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        """
        Calcula a entropia marginal (média sobre o batch).
        """
        # Implementação padrão: média dos parâmetros no batch
        avg_dist_info = {
            k: tf.tile(tf.reduce_mean(v, axis=0, keepdims=True), 
            [tf.shape(v)[0], *[1]*(len(v.shape)-1)])
            for k, v in dist_info.items()
        }
        return self.entropy(avg_dist_info)
        
    def marginal_logli(self, x_var: tf.Tensor, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        """
        Log-likelihood usando parâmetros marginais (média no batch).
        """
        avg_dist_info = {
            k: tf.tile(tf.reduce_mean(v, axis=0, keepdims=True), 
            [tf.shape(v)[0], *[1]*(len(v.shape)-1)])
            for k, v in dist_info.items()
        }
        return self.logli(x_var, avg_dist_info)
        
    def kl(self, p: Dict[str, tf.Tensor], q: Dict[str, tf.Tensor]) -> tf.Tensor:
        """
        Calcula a divergência KL entre duas distribuições (KL(p||q)).
        Implementação padrão usando entropia cruzada e entropia:
        KL(p||q) = H(p,q) - H(p)
        """
        cross_entropy = -self.logli(p['samples'], q)
        entropy = self.entropy(p)
        return cross_entropy - entropy
        
    def dist_info_keys(self) -> List[str]:
        """
        Lista de chaves no dicionário de parâmetros da distribuição.
        """
        raise NotImplementedError
        
    def activate_dist(self, flat_dist: tf.Tensor) -> Dict[str, tf.Tensor]:
        """
        Converte um vetor plano de parâmetros em parâmetros da distribuição.
        """
        raise NotImplementedError
        
    def nonreparam_logli(self, x_var: tf.Tensor, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        """
        Log-likelihood para distribuições sem reparameterization trick.
        """
        return tf.zeros_like(x_var[:, 0])

class Categorical(Distribution):
    """Distribuição categórica (one-hot) para variáveis latentes discretas."""
    
    def __init__(self, dim: int):
        self._dim = dim
        
    @property
    def dim(self) -> int:
        return self._dim
        
    @property
    def dist_flat_dim(self) -> int:
        return self._dim
        
    @property
    def effective_dim(self) -> int:
        return 1  # Apesar de ter N categorias, a dimensão efetiva é 1
        
    def logli(self, x_var: tf.Tensor, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        prob = dist_info['prob']
        return tf.reduce_sum(x_var * tf.math.log(prob + TINY), axis=1)
        
    def sample(self, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        prob = dist_info['prob']
        # Gera amostras usando Gumbel-Softmax trick para diferenciabilidade
        logits = tf.math.log(prob + TINY)
        gumbel_noise = -tf.math.log(-tf.math.log(tf.random.uniform(tf.shape(logits), dtype=floatX)))
        samples = tf.nn.softmax((logits + gumbel_noise) / 1.0)  # temperatura=1.0
        return samples
        
    def prior_dist_info(self, batch_size: int) -> Dict[str, tf.Tensor]:
        prob = tf.ones((batch_size, self.dim), dtype=floatX) / self.dim
        return {'prob': prob}
        
    def entropy(self, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        prob = dist_info['prob']
        return -tf.reduce_sum(prob * tf.math.log(prob + TINY), axis=1)
        
    def kl(self, p: Dict[str, tf.Tensor], q: Dict[str, tf.Tensor]) -> tf.Tensor:
        p_prob, q_prob = p['prob'], q['prob']
        return tf.reduce_sum(p_prob * (tf.math.log(p_prob + TINY) - tf.math.log(q_prob + TINY)), axis=1)
        
    def dist_info_keys(self) -> List[str]:
        return ['prob']
        
    def activate_dist(self, flat_dist: tf.Tensor) -> Dict[str, tf.Tensor]:
        return {'prob': tf.nn.softmax(flat_dist)}

class Gaussian(Distribution):
    """Distribuição gaussiana para variáveis latentes contínuas."""
    
    def __init__(self, dim: int, fix_std: bool = False):
        self._dim = dim
        self._fix_std = fix_std
        
    @property
    def dim(self) -> int:
        return self._dim
        
    @property
    def dist_flat_dim(self) -> int:
        return self._dim * 2 if not self._fix_std else self._dim
        
    def logli(self, x_var: tf.Tensor, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        mean, std = dist_info['mean'], dist_info['stddev']
        z = (x_var - mean) / (std + TINY)
        return tf.reduce_sum(-0.5 * (np.log(2 * np.pi) + tf.math.log(std + TINY) + 0.5 * tf.square(z)), axis=1)
        
    def sample(self, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        mean, std = dist_info['mean'], dist_info['stddev']
        return mean + std * tf.random.normal(tf.shape(mean))
        
    def prior_dist_info(self, batch_size: int) -> Dict[str, tf.Tensor]:
        mean = tf.zeros((batch_size, self.dim), dtype=floatX)
        std = tf.ones((batch_size, self.dim), dtype=floatX)
        return {'mean': mean, 'stddev': std}
        
    def entropy(self, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        std = dist_info['stddev']
        return tf.reduce_sum(0.5 * np.log(2 * np.pi * np.e) + tf.math.log(std + TINY), axis=1)
        
    def kl(self, p: Dict[str, tf.Tensor], q: Dict[str, tf.Tensor]) -> tf.Tensor:
        p_mean, p_std = p['mean'], p['stddev']
        q_mean, q_std = q['mean'], q['stddev']
        return tf.reduce_sum(
            tf.math.log(q_std + TINY) - tf.math.log(p_std + TINY) + 
            (tf.square(p_std) + tf.square(p_mean - q_mean)) / (2 * tf.square(q_std + TINY)) - 0.5,
            axis=1
        )
        
    def dist_info_keys(self) -> List[str]:
        return ['mean', 'stddev']
        
    def activate_dist(self, flat_dist: tf.Tensor) -> Dict[str, tf.Tensor]:
        mean = flat_dist[:, :self.dim]
        if self._fix_std:
            std = tf.ones_like(mean)
        else:
            std = tf.sqrt(tf.exp(flat_dist[:, self.dim:]))
        return {'mean': mean, 'stddev': std}

class Product(Distribution):
    """Produto de distribuições para combinar diferentes tipos de variáveis latentes."""
    
    def __init__(self, dists: List[Distribution]):
        self._dists = dists
        
    @property
    def dim(self) -> int:
        return sum(d.dim for d in self._dists)
        
    @property
    def dist_flat_dim(self) -> int:
        return sum(d.dist_flat_dim for d in self._dists)
    @property
    def dists(self):  
        return self._dists
    
    @property
    def effective_dim(self) -> int:
        return sum(d.effective_dim for d in self._dists)
        
    def split_dist_info(self, dist_info: Dict[str, tf.Tensor]) -> List[Dict[str, tf.Tensor]]:
        """Divide um dicionário de parâmetros combinado em dicionários por distribuição."""
        split_infos = []
        for i, dist in enumerate(self._dists):
            info = {}
            for key in dist.dist_info_keys():
                info[key] = dist_info[f'dist{i}_{key}']
            split_infos.append(info)
        return split_infos
        
    def join_dist_infos(self, dist_infos: List[Dict[str, tf.Tensor]]) -> Dict[str, tf.Tensor]:
        """Combina dicionários de parâmetros em um único dicionário."""
        joint_info = {}
        for i, (dist, info) in enumerate(zip(self._dists, dist_infos)):
            for key in dist.dist_info_keys():
                joint_info[f'dist{i}_{key}'] = info[key]
        return joint_info
        
    def logli(self, x_var: tf.Tensor, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        split_infos = self.split_dist_info(dist_info)
        dims = [d.dim for d in self._dists]
        split_x = tf.split(x_var, dims, axis=1)
        return tf.add_n([d.logli(x, i) for d, x, i in zip(self._dists, split_x, split_infos)])
        
    def sample(self, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        split_infos = self.split_dist_info(dist_info)
        samples = [d.sample(i) for d, i in zip(self._dists, split_infos)]
        return tf.concat(samples, axis=1)
        
    def prior_dist_info(self, batch_size: int) -> Dict[str, tf.Tensor]:
        dist_infos = [d.prior_dist_info(batch_size) for d in self._dists]
        return self.join_dist_infos(dist_infos)
        
    def entropy(self, dist_info: Dict[str, tf.Tensor]) -> tf.Tensor:
        split_infos = self.split_dist_info(dist_info)
        return tf.add_n([d.entropy(i) for d, i in zip(self._dists, split_infos)])
        
    def kl(self, p: Dict[str, tf.Tensor], q: Dict[str, tf.Tensor]) -> tf.Tensor:
        p_split = self.split_dist_info(p)
        q_split = self.split_dist_info(q)
        return tf.add_n([d.kl(pi, qi) for d, pi, qi in zip(self._dists, p_split, q_split)])
        
    def dist_info_keys(self) -> List[str]:
        keys = []
        for i, dist in enumerate(self._dists):
            for key in dist.dist_info_keys():
                keys.append(f'dist{i}_{key}')
        return keys
        
    def activate_dist(self, flat_dist: tf.Tensor) -> Dict[str, tf.Tensor]:
        dist_infos = []
        sizes = [d.dist_flat_dim for d in self._dists]
        split_flat = tf.split(flat_dist, sizes, axis=1)
        for dist, flat in zip(self._dists, split_flat):
            dist_infos.append(dist.activate_dist(flat))
        return self.join_dist_infos(dist_infos)

    def prior_dist_info(self, batch_size: int) -> Dict[str, tf.Tensor]:
        infos = [d.prior_dist_info(batch_size) for d in self._dists]  
        return self.join_dist_infos(infos)


# Modelos Keras 
#Gerador DCGAN-64: projeção → reshape → 4 transposed‐convs com batch norm + ReLU, final em tanh.
def build_generator(z_dim: int, img_shape):
    """Gerador DCGAN‑64 clássico (4× upsampling → 64×64)."""
    h, w, c = img_shape  # h==w==64

    inp = layers.Input(shape=(z_dim,))

    # 1) projeção + reshape → 4×4×512
    x = layers.Dense(4 * 4 * 512, use_bias=False)(inp)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Reshape((4, 4, 512))(x)

    # 2) 8×8×256
    x = layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # 3) 16×16×128
    x = layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # 4) 32×32×64
    x = layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # 5) 64×64×c
    x = layers.Conv2DTranspose(c, kernel_size=4, strides=2, padding='same', activation='tanh')(x)

    return Model(inp, x, name='Generator')


def build_discriminator_q(img_shape, cat_dim, cont_dim):
    inp = layers.Input(shape=img_shape)
    x = layers.Conv2D(64, 4, 2, 'same')(inp)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Conv2D(128, 4, 2, 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1024)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    # discriminator
    d_out = layers.Dense(1, activation='sigmoid', name='d_out')(x)

    # Q‐network: duas saídas
    q_cat_logits = layers.Dense(cat_dim, name='q_cat_logits')(x)
    q_cont_params = layers.Dense(cont_dim * 2, name='q_cont_params')(x)

    return Model(inp, [d_out, q_cat_logits, q_cont_params], name='Discriminator_Q')


# FID
_inception = tf.keras.applications.InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

FID_BATCH = 1024  

def _get_inception_activations(img_uint8, bs: int = FID_BATCH):
    """Extrai ativações do pool-3 da Inception em minibatches para não estourar RAM."""
    acts = []
    for i in range(0, len(img_uint8), bs):
        batch = img_uint8[i:i + bs]
        batch = tf.image.resize(batch, (299, 299))
        batch = tf.keras.applications.inception_v3.preprocess_input(tf.cast(batch, tf.float32))
        acts.append(_inception(batch, training=False))
    return tf.concat(acts, axis=0).numpy()

def fid_np(real_uint8, gen_uint8):
    act1, act2 = _get_inception_activations(real_uint8), _get_inception_activations(gen_uint8)
    mu1, mu2 = act1.mean(0), act2.mean(0)
    sigma1, sigma2 = np.cov(act1, rowvar=False), np.cov(act2, rowvar=False)
    covmean = sqrtm(sigma1 @ sigma2)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    return float(np.sum((mu1 - mu2) ** 2) + np.trace(sigma1 + sigma2 - 2 * covmean))

2025-07-17 21:33:39.290126: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-17 21:33:39.299604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752798819.311002 2958359 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752798819.314107 2958359 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752798819.322582 2958359 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [5]:
#métricas
def hsic(x: tf.Tensor, y: tf.Tensor):
    """Calcula o Hilbert-Schmidt Independence Criterion (HSIC)"""
    m = tf.cast(tf.shape(x)[0], tf.float32)
    K = tf.matmul(x, x, transpose_b=True)
    L = tf.matmul(y, y, transpose_b=True)
    H = tf.eye(m) - tf.ones((m, m)) / m
    KH = tf.matmul(K, H)
    LH = tf.matmul(L, H)
    return tf.linalg.trace(tf.matmul(KH, LH)) / (m * m)

def mig(codes, factors):
    """Calcula o Mutual Information Gap (MIG)"""
    # Simplificada
    mi_matrix = np.zeros((factors.shape[1], codes.shape[1]))
    for i in range(factors.shape[1]):
        for j in range(codes.shape[1]):
            mi_matrix[i,j] = mutual_info_score(factors[:,i], codes[:,j])
    
    sorted_mi = np.sort(mi_matrix, axis=0)[::-1]
    return np.mean((sorted_mi[0,:] - sorted_mi[1,:]) / sorted_mi[0,:])

def sap_score(codes, factors):
    """Calcula o Separated Attribute Predictability (SAP)"""
    # Simplificada
    scores = []
    for i in range(factors.shape[1]):
        pred_scores = []
        for j in range(codes.shape[1]):
            pred_scores.append(np.abs(pearsonr(factors[:,i], codes[:,j])[0]))
        scores.append(np.mean(pred_scores))
    return np.mean(scores)

def quase_ortogo(z: tf.Tensor, eps=1e-5):
    """Calcula métricas de quase-ortogonalidade para códigos latentes"""
    # 1. Normalizar os vetores
    zn = tf.math.l2_normalize(z, axis=0)
    
    # 2. Verificar desvio padrão
    std = tf.math.reduce_std(zn, axis=0)
    if tf.reduce_any(std < 1e-6):
        return False, float("nan"), float("nan"), float("nan")
    
    # 3. Calcular matriz de Gram
    G = tf.matmul(zn, zn, transpose_a=True)
    
    # 4. Zerar diagonal
    mask = tf.eye(tf.shape(G)[0], dtype=tf.bool)
    G = tf.where(mask, tf.zeros_like(G), G)
    
    # 5. Calcular correlações máximas e mínimas
    max_corr = tf.reduce_max(tf.abs(G))
    min_corr = tf.reduce_min(tf.abs(G))
    
    # 6. Calcular proporção de pares abaixo de epsilon
    below_eps = tf.reduce_sum(tf.cast(tf.abs(G) < eps, dtype=tf.float32))
    total_pairs = tf.cast(tf.size(G) - tf.shape(G)[0], tf.float32)
    prop = below_eps / total_pairs
    
    return min_corr.numpy(), max_corr.numpy(), prop.numpy()


In [6]:
from sklearn.metrics import mutual_info_score
from scipy.stats import pearsonr

# InfoGAN Trainer 
class InfoGANTrainer:
    def __init__(self, G, DQ, latent_dist: Product, dataset: CelebADataset, batch_size=64,
             info_coeff=1.0, log_dir='logs', ckpt_dir='ckpt', snapshot=1000, max_iter=100_000,
             noise_dim=62, cat_dim=10, cont_dim=2):
        self.G, self.DQ = G, DQ
        self.latent_dist, self.dataset = latent_dist, dataset
        self.bs = batch_size
        self.info_coeff = info_coeff
        self.snapshot = snapshot
        self.max_iter = max_iter
        self.noise_dim = noise_dim
        self.cat_dim   = cat_dim
        self.cont_dim  = cont_dim

        self.log_dir, self.ckpt_dir = Path(log_dir), Path(ckpt_dir)
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            try:
                tf.config.experimental.set_memory_growth(gpus[0], True)
                tf.config.optimizer.set_jit(True)  # Ativa XLA
            except RuntimeError as e:
                print(f"Erro ao configurar GPU: {e}")
        else:
            print("Executando em CPU")
            
        mkdir_p(self.log_dir); mkdir_p(self.ckpt_dir)

        self.d_opt = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
        self.g_opt = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
        self.q_opt = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
        
        tf.config.optimizer.set_jit(True)  
        #tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

        # Filtra os pesos da Q-head pelo nome
        self.q_vars = [
            v for v in self.DQ.trainable_variables
            if 'q_cat_logits' in v.name or 'q_cont_params' in v.name
        ]
        
        self.ckpt = tf.train.Checkpoint(
            generator=self.G,
            discriminator=self.DQ,
            g_optimizer=self.g_opt,
            d_optimizer=self.d_opt,
            q_optimizer=self.q_opt
        )
        self.ckpt_manager = tf.train.CheckpointManager(
            self.ckpt,
            directory=str(self.ckpt_dir),
            max_to_keep=5
        )
        
        # Métricas 
        self.metric_hist = {
            'iter': [],
            'FID': [],
            'HSIC': [],
            'MIG': [],
            'SAP': [],
            'Ortho_min': [],
            'Ortho_max': [],
            'Ortho_prop': [],
            'D_loss': [],
            'G_loss': [],
            'MI_loss': []
        }
        
        self.metric_path = self.log_dir / 'metricsGAN.csv'
        
        # Restaurar o último checkpoint se existir
        self.start_iter = 0
        if self.ckpt_manager.latest_checkpoint:
            self.ckpt.restore(self.ckpt_manager.latest_checkpoint)
            print(f"Checkpoint restaurado: {self.ckpt_manager.latest_checkpoint}")
            # Extrair o número da iteração do nome do checkpoint
            import re
            match = re.search(r'ckpt-(\d+)', self.ckpt_manager.latest_checkpoint)
            if match:
                self.start_iter = int(match.group(1))
                print(f"Continuando do passo {self.start_iter}")

    def _mi_loss(self, z_reg, q_cat_logits, q_cont_params):
        # separa z_reg em z_cat (one-hot) e z_cont (gaussiano)
        z_cat  = z_reg[:, :self.cat_dim]
        z_cont = z_reg[:, self.cat_dim:]

        # 1) Cross-entropy (categórico)
        cat_loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                labels=z_cat,
                logits=q_cat_logits
            )
        )

        # 2) Log-likelihood gaussiano (contínuo)
        mean_pred    = q_cont_params[:, :self.cont_dim]
        log_std_pred = q_cont_params[:, self.cont_dim:]
        std_pred     = tf.exp(log_std_pred)

        eps = (z_cont - mean_pred) / (std_pred + TINY)
        logli_per_dim = -0.5 * (tf.math.log(2. * np.pi) + 2. * log_std_pred + tf.square(eps))
        logli = tf.reduce_sum(logli_per_dim, axis=1)
        cont_loss = -tf.reduce_mean(logli)  

        return cat_loss + cont_loss

    def _evaluate_metrics(self, n_samples=1000):
        """Calcula todas as métricas usando um conjunto de amostras"""
        # Gerar amostras
        real_flat = self.dataset.next_batch(n_samples)
        z = self.latent_dist.sample_prior(n_samples)
        gen_imgs = self.G(z, training=False).numpy()
        
        # 1. FID
        #fid = fid_np(self.dataset.inverse_transform(real_flat),self.dataset.inverse_transform(gen_imgs.reshape(real_flat.shape))
        
        # 2. HSIC entre variáveis latentes
        hsic_val = hsic(z[:, :self.noise_dim], z[:, self.noise_dim:]).numpy()
        
        # 3. MIG e SAP 
        
        try:
            mig_val = mig(z.numpy(), real_flat.numpy())
            sap_val = sap_score(z.numpy(), real_flat.numpy())
        except:
            mig_val, sap_val = float('nan'), float('nan')
        
        # 4. Quase-ortogonalidade
        ortho_min, ortho_max, ortho_prop = quase_orto(z)
            
        return {
            'HSIC': hsic_val,
            'MIG': mig_val,
            'SAP': sap_val,
            'Ortho_min': ortho_min,
            'Ortho_max': ortho_max,
            'Ortho_prop': ortho_prop
        }
            
            
    @tf.function
    def _train_step(self, real_imgs):
        # 1) Amostra z
        z = self.latent_dist.sample_prior(self.bs)

        # Atualiza Discriminador
        with tf.GradientTape() as d_tape:
            fake = self.G(z, training=True)
            d_real, q_cat_logits, q_cont_params = self.DQ(real_imgs, training=True)
            d_fake, _, _ = self.DQ(fake, training=True)
            d_loss = -tf.reduce_mean(tf.math.log(d_real + TINY) + tf.math.log(1 - d_fake + TINY))

        # Calcula gradientes apenas para o Discriminador (excluindo Q)
        d_vars = [v for v in self.DQ.trainable_variables 
                  if 'q_cat_logits' not in v.name and 'q_cont_params' not in v.name]
        d_grads = d_tape.gradient(d_loss, d_vars)
        self.d_opt.apply_gradients(zip(d_grads, d_vars))

        # Atualiza Gerador e Rede Q juntos
        with tf.GradientTape(persistent=True) as g_tape:
            fake = self.G(z, training=True)
            d_fake, q_cat_logits, q_cont_params = self.DQ(fake, training=True)

            # Loss adversarial
            g_adv_loss = -tf.reduce_mean(tf.math.log(d_fake + TINY))

            # Loss de informação mútua
            z_reg = z[:, self.noise_dim:]
            mi_loss = self._mi_loss(z_reg, q_cat_logits, q_cont_params)

            g_loss = g_adv_loss + self.info_coeff * mi_loss

        # Gradientes para o Gerador
        g_grads = g_tape.gradient(g_loss, self.G.trainable_variables)
        self.g_opt.apply_gradients(zip(g_grads, self.G.trainable_variables))

        # Gradientes para a Rede Q (apenas MI loss)
        q_grads = g_tape.gradient(mi_loss, self.q_vars)
        self.q_opt.apply_gradients(zip(q_grads, self.q_vars))

        del g_tape  

        return d_loss, g_adv_loss, mi_loss

    def train(self):
        for step in tqdm(range(self.start_iter + 1, self.max_iter + 1), 
                        initial=self.start_iter + 1, 
                        total=self.max_iter):
            real_flat = self.dataset.next_batch(self.bs)
            real_imgs = real_flat.reshape((-1,) + self.dataset.image_shape)
            d_loss, g_loss, mi = self._train_step(real_imgs)

            if step % self.snapshot == 0:
                # Avaliar todas as métricas
                metrics = self._evaluate_metrics()
                
                # Atualizar histórico de métricas
                self.metric_hist['iter'].append(step)
                self.metric_hist['FID'].append(metrics['FID'])
                self.metric_hist['HSIC'].append(metrics['HSIC'])
                self.metric_hist['MIG'].append(metrics['MIG'])
                self.metric_hist['SAP'].append(metrics['SAP'])
                self.metric_hist['Ortho_min'].append(metrics['Ortho_min'])
                self.metric_hist['Ortho_max'].append(metrics['Ortho_max'])
                self.metric_hist['Ortho_prop'].append(metrics['Ortho_prop'])
                self.metric_hist['D_loss'].append(float(d_loss))
                self.metric_hist['G_loss'].append(float(g_loss))
                self.metric_hist['MI_loss'].append(float(mi))
                
                # Salvar métricas
                pd.DataFrame(self.metric_hist).to_csv(self.metric_path, index=False)
                
                # Salvar checkpoint
                self.ckpt_manager.save(checkpoint_number=step)
                
                # Gerar amostras visuais
                sample_dir = self.log_dir / f'samples_step_{step:06d}'
                self.generate_samples(n_samples=16, output_dir=str(sample_dir))
                
                print(f"\nStep {step}:")
                
                print(f"  HSIC: {metrics['HSIC']:.4f}")
                print(f"  MIG: {metrics['MIG']:.4f}")
                print(f"  SAP: {metrics['SAP']:.4f}")
                print(f"  Ortho: min={metrics['Ortho_min']:.4f}, max={metrics['Ortho_max']:.4f}, prop={metrics['Ortho_prop']:.4f}")
                print(f"  Losses: D={d_loss:.4f}, G={g_loss:.4f}, MI={mi:.4f}")
                print(f"  Checkpoint salvo em {self.ckpt_dir}/ckpt-{step}")
                print(f"  Amostras salvas em {sample_dir}")
            
            if step + 1 == self.max_iter:
                # Gerar amostras
                real_flat = self.dataset.next_batch(n_samples)
                z = self.latent_dist.sample_prior(n_samples)
                gen_imgs = self.G(z, training=False).numpy()

                # 1. FID
                fid = fid_np(
                    self.dataset.inverse_transform(real_flat),
                    self.dataset.inverse_transform(gen_imgs.reshape(real_flat.shape)))
                print(f"FID: {fid}")



    def load_latest_checkpoint(self):
        """Carrega o último checkpoint disponível"""
        if self.ckpt_manager.latest_checkpoint:
            self.ckpt.restore(self.ckpt_manager.latest_checkpoint)
            print(f"Checkpoint carregado: {self.ckpt_manager.latest_checkpoint}")
            return True
        print("Nenhum checkpoint encontrado para carregar")
        return False

    def generate_samples(self, n_samples: int = 8, output_dir: str = 'samples', prefix: str = 'sample'):
        """Gera e salva amostras como imagens PNG."""
        mkdir_p(output_dir)
        z = self.latent_dist.sample_prior(n_samples)
        gen = self.G(z, training=False).numpy()
        imgs_uint8 = self.dataset.inverse_transform(gen.reshape(n_samples, -1))

        for i, img in enumerate(imgs_uint8):
            path = os.path.join(output_dir, f"{prefix}_{i}.png")
            Image.fromarray(img).save(path)

        print(f"Geradas e salvas {n_samples} amostras em '{output_dir}/'.")

In [7]:
BATCH = 64
IMG_SHAPE = (64, 64, 3)
    
#Parametros para latente (dimensão do vetor \Re^(noise_dim + cat_dim + cont_dim))
noise_dim, cat_dim, cont_dim = 62, 10, 2
    
# O espaço latente modelado pelo objeto Product que agrupa três distribuições:
latent_dist = Product([Gaussian(noise_dim, fix_std=True),Categorical(cat_dim),Gaussian(cont_dim)])

G = build_generator(noise_dim + cat_dim + cont_dim, IMG_SHAPE)
DQ = build_discriminator_q(IMG_SHAPE, cat_dim, cont_dim)
data = CelebADataset('data/celeba', IMG_SHAPE)

trainer = InfoGANTrainer(G, DQ, latent_dist, data, batch_size=BATCH, max_iter=100000, snapshot=100, 
                             noise_dim=noise_dim,cat_dim=cat_dim, cont_dim=cont_dim)
trainer.train()

Executando em CPU


  0%|          | 1/100000 [00:00<?, ?it/s]/home/hanna/Github/mod_gen/.venv/lib/python3.9/site-packages/keras/src/optimizers/base_optimizer.py:855: UserWarning: Gradients do not exist for variables ['q_cat_logits/kernel', 'q_cat_logits/bias', 'q_cont_params/kernel', 'q_cont_params/bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(
  0%|          | 1/100000 [00:01<?, ?it/s]


ValueError: in user code:

    File "/tmp/ipykernel_2958359/4198870918.py", line 184, in _train_step  *
        self.q_opt.apply_gradients(zip(q_grads, self.q_vars))
    File "/home/hanna/Github/mod_gen/.venv/lib/python3.9/site-packages/keras/src/optimizers/base_optimizer.py", line 462, in apply_gradients  **
        grads, trainable_variables = zip(*grads_and_vars)

    ValueError: not enough values to unpack (expected 2, got 0)
